# the function to draw 3*3 plot

In [1]:
import warnings

warnings.simplefilter("ignore")
from dipy.reconst.dti import fractional_anisotropy, color_fa
from argparse import ArgumentParser
from scipy import ndimage
import os
import re
import numpy as np
import nibabel as nb
import sys
import matplotlib
from PIL import Image

matplotlib.use("Agg")  # very important above pyplot import
import matplotlib.pyplot as plt
import matplotlib.image as mping


def plot_rgb(im):
    plt.rcParams.update({"axes.labelsize": "x-large", "axes.titlesize": "x-large"})

    if im.shape == (182, 218, 182):
        x = [78, 90, 100]
        y = [82, 107, 142]
        z = [88, 103, 107]
    else:
        shap = im.shape
        x = [int(shap[0] * 0.35), int(shap[0] * 0.51), int(shap[0] * 0.65)]
        y = [int(shap[1] * 0.35), int(shap[1] * 0.51), int(shap[1] * 0.65)]
        z = [int(shap[2] * 0.35), int(shap[2] * 0.51), int(shap[2] * 0.65)]
    coords = (x, y, z)

    labs = [
        "Sagittal Slice (YZ fixed)",
        "Coronal Slice (XZ fixed)",
        "Axial Slice (XY fixed)",
    ]
    var = ["X", "Y", "Z"]

    idx = 0
    for i, coord in enumerate(coords):
        for pos in coord:
            idx += 1
            ax = plt.subplot(3, 3, idx)
            ax.set_title(var[i] + " = " + str(pos))
            if i == 0:
                image = ndimage.rotate(im[pos, :, :], 90)
            elif i == 1:
                image = ndimage.rotate(im[:, pos, :], 90)
            else:
                image = im[:, :, pos]

            if idx % 3 == 1:
                ax.set_ylabel(labs[i])
                ax.yaxis.set_ticks([0, image.shape[0] / 2, image.shape[0] - 1])
                ax.xaxis.set_ticks([0, image.shape[1] / 2, image.shape[1] - 1])

            plt.imshow(image)
#             image.show()


    fig = plt.gcf()
    fig.set_size_inches(12.5, 10.5, forward=True)
    return fig

# QA of witer matter, gray matter and CSF

In [2]:
m1 = 'd:/Downloads/neurodatadesign/output_data/flirt/sub-0025864/ses-1/anat/preproc/t1w_seg_pve_0.nii.gz'
m2 = 'd:/Downloads/neurodatadesign/output_data/flirt/sub-0025864/ses-1/anat/preproc/t1w_seg_pve_1.nii.gz'
m3 = 'd:/Downloads/neurodatadesign/output_data/flirt/sub-0025864/ses-1/anat/preproc/t1w_seg_pve_2.nii.gz'
n1 = nb.load(m1)
n2 = nb.load(m2)
n3 = nb.load(m3)
im1 = n1.get_data()
im2 = n2.get_data()
im3 = n3.get_data()
fig1 = plot_rgb(im1)
rect = fig1.patch
fig1.show()
plt.savefig('e:/brain_wm.png', format="png")
fig2 = plot_rgb(im2)
plt.savefig('e:/brain_gm.png', format="png")
fig3 = plot_rgb(im3)
plt.savefig('e:/brain_csf.png', format="png")

# the function of overlay picture

In [4]:
def overlay_viz(wm_input, gm_input, csf_input):
    wm_RGB = Image.open(wm_input).convert('RGB')
    gm_RGB = Image.open(gm_input).convert('RGB')
    csf_RGB = Image.open(csf_input).convert('RGB')
    r1, g1, b1 = wm_RGB.split()
    r2, g2, b2 = gm_RGB.split()
    r3, g3, b3 = csf_RGB.split()
    pic = Image.merge('RGB', (r3, g1, b2))
    pic.save('e:/overlay_pic.png')

overlay_viz('e:/brain_wm.png','e:/brain_gm.png','e:/brain_csf.png')